# Start

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import numpy as np
from ai4ha.GAN.Generators import WaveGANGenerator, Pulse2PulseGANGenerator, WaveGANUpsampleGenerator, \
    WaveGANDeconvGenerator, TTSGANGenerator

from ai4ha.GAN.Discriminators import WaveGANDiscriminator, TTSGANDiscriminator
from ai4ha.util import instantiate_from_config, load_config

In [6]:
data = torch.rand(10, 8, 512).to('cuda')

In [7]:
gen = Pulse2PulseGANGenerator(
        seq_length=512,
        seq_channels=8,
        init_channels=32,
        kernel_size=16,
        stride=2,
        n_layers=3,
        n_classes=0,
        batch_norm=True,)

/home/bejar/env/torch13/lib64/python3.10/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [8]:
gen.to('cuda');

Pulse2PulseGANGenerator(
  (class_adapt): Linear(in_features=0, out_features=512, bias=True)
  (convolutions): ModuleList(
    (0): Sequential(
      (0): Conv1d(8, 32, kernel_size=(16,), stride=(2,), padding=(7,))
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(32, 64, kernel_size=(16,), stride=(2,), padding=(7,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv1d(64, 128, kernel_size=(16,), stride=(2,), padding=(7,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): Conv1d(128, 256, kernel_size=(16,), stride=(2,), padding=(7,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (first_deconvolution): Sequential(


In [5]:
out = gen(data)

In [5]:
out.shape

torch.Size([10, 8, 512])

---

In [2]:
from ai4ha.layers.Layers import UpsamplingConvo1D

In [67]:
data = torch.rand(10, 8, 32).to('cuda')

In [83]:
upc = UpsamplingConvo1D(in_channels=8, out_channels=16, upsample=4, kernel_size=3, stride=1, padding=0).to('cuda')

In [84]:
res = upc(data)

In [85]:
res.shape

torch.Size([10, 16, 128])

---

In [14]:
config = load_config('configs/done/Pulse2Pulse-MITBIH-G-NL4-IC64-K16-S2-BN-D-NL3-IC64-K16-S2-SF8-SN-local.json')

In [15]:
netG = instantiate_from_config(config['model']['generator'])

In [16]:
path = '/home/bejar/bsc/misiones/Series/Models/WaveGAN/logs/Pulse2Pulse-MITBIH-G-NL4-IC64-K16-S2-BN-D-NL3-IC64-K16-S2-SF8-SN/final/Generator/pytorch_model.bin'
netG.load_state_dict(torch.load(path))

RuntimeError: Error(s) in loading state_dict for Pulse2PulseGANGenerator:
	Unexpected key(s) in state_dict: "convolutions.0.1.weight", "convolutions.0.1.bias", "convolutions.0.1.running_mean", "convolutions.0.1.running_var", "convolutions.0.1.num_batches_tracked", "convolutions.1.1.weight", "convolutions.1.1.bias", "convolutions.1.1.running_mean", "convolutions.1.1.running_var", "convolutions.1.1.num_batches_tracked", "convolutions.2.1.weight", "convolutions.2.1.bias", "convolutions.2.1.running_mean", "convolutions.2.1.running_var", "convolutions.2.1.num_batches_tracked", "convolutions.3.1.weight", "convolutions.3.1.bias", "convolutions.3.1.running_mean", "convolutions.3.1.running_var", "convolutions.3.1.num_batches_tracked", "convolutions.4.1.weight", "convolutions.4.1.bias", "convolutions.4.1.running_mean", "convolutions.4.1.running_var", "convolutions.4.1.num_batches_tracked", "deconvolutions.0.1.weight", "deconvolutions.0.1.bias", "deconvolutions.0.1.running_mean", "deconvolutions.

In [10]:
netG.eval()

AttributeError: 'dict' object has no attribute 'eval'

---

In [2]:
data = torch.rand(10, 100).to('cuda')
labels = torch.randint(0, 2, (10,)).to('cuda')

In [3]:
gen = WaveGANDeconvGenerator(latent_dim=100,
                               init_dim=16,
                               init_channels=8,
                               output_size=256,
                               output_channels=1,
                               kernel_size=15,
                               stride=2,
                               n_classes=5,
                               class_embedding_dim=10,
                               normalization="batch",
                               upsampling=True)

gen.to('cuda');

In [4]:
res = gen(data, labels)

In [5]:
res.shape

torch.Size([10, 1, 256])

In [8]:
disc = WaveGANDiscriminator(
    input_size=256,
    input_channels=8,
    init_channels=16,
    n_layers=6,
    kernel_size=4,
    stride=2,
    shift_factor=2)

In [7]:
disc(res)

torch.Size([1, 8, 256])
torch.Size([1, 16, 128])
torch.Size([1, 16, 128])
torch.Size([1, 16, 128])
torch.Size([1, 32, 64])
torch.Size([1, 32, 64])
torch.Size([1, 32, 64])
torch.Size([1, 64, 32])
torch.Size([1, 64, 32])


tensor([[0.4887]], grad_fn=<SigmoidBackward0>)

---

In [2]:
data = torch.rand(10, 8, 512).to('cuda')
labels = torch.randint(0, 5, (10,)).to('cuda')

In [3]:
labels

tensor([3, 0, 4, 1, 0, 1, 3, 0, 2, 3], device='cuda:0')

In [4]:
gen = Pulse2PulseGANGenerator(
    seq_length=512,
    seq_channels=8,
    init_channels=32,
    kernel_size=16,
    stride=2,
    n_layers=3,
    n_classes=5,
    class_embedding=True,
    normalization=None,
)

gen.to('cuda');

In [5]:
res = gen(data, labels)

In [6]:
res.shape

torch.Size([10, 8, 512])

# TTSGAN

In [2]:
data = torch.rand(10, 100).to('cuda')
labels = torch.randint(0, 5, (10,)).to('cuda')

In [3]:
gen = TTSGANGenerator(
        latent_dim=100,
        seq_length=512,
        seq_channels=8,
        n_classes=5,
        data_embed_dim=128,
        label_embed_dim=10,
        n_layers=3,
        num_heads=8,
        dropout_rate=0.5,
)

gen.to('cuda');

In [4]:
res = gen(data, labels)

In [5]:
res.shape

torch.Size([10, 8, 512])

In [6]:
dis = TTSGANDiscriminator(
    seq_length=512,
    seq_channels=8,
    n_classes=5,
    patch_size=16,
    data_embed_dim=64,
    label_embed_dim=10,
    n_layers=3,
    num_heads=8,
    dropout_rate=0.5,
)
dis.to('cuda');

In [7]:
dis(res, labels)

tensor([[ 0.5648],
        [-3.1616],
        [ 0.4286],
        [ 1.0198],
        [ 4.8022],
        [-1.5704],
        [ 1.1968],
        [ 1.5078],
        [ 1.3790],
        [-0.3047]], device='cuda:0', grad_fn=<AddBackward0>)

# MAE

In [59]:
data = torch.rand(10, 2, 128).to('cuda')
data2= torch.rand(10, 2, 128).to('cuda')
labels = torch.randint(0, 5, (10, )).to('cuda')
patch = 8

In [60]:
patchl = nn.Conv1d(2, 4, kernel_size=patch, stride=patch).to('cuda')
linear = nn.Conv1d(4, 1, kernel_size=1, stride=1).to('cuda')

In [62]:
x = patchl(data)
x2 = patchl(data2)

In [49]:
x.shape

torch.Size([10, 4, 16])

In [63]:
r = linear(x)
r2 = linear(x2)

In [46]:
r.shape

torch.Size([10, 1, 16])

In [57]:
# r = r.view(r.shape[0], -1)

In [64]:
(torch.log(torch.sigmoid(r))+torch.log(1-torch.sigmoid(r2))).mean()

tensor(-1.3855, device='cuda:0', grad_fn=<MeanBackward0>)